In [1]:
from numpy import mean
from numpy import std
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.decomposition import PCA
import copy
from sklearn.preprocessing import StandardScaler
import sklearn.feature_selection
from sklearn.linear_model import LogisticRegression

In [2]:
trainX = pd.read_csv('X_Train.csv')
trainY = pd.read_csv('Y_train.csv')
testX = pd.read_csv('X_test.csv')

In [3]:
trainX = trainX.drop(columns=['id', 'patient_no'])
testX = testX.drop(columns=['id', 'patient_no'])

In [4]:
array = ['cast', 'gender', 'age group', 'weight', 'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 
                                           'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 
                                            'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 
                                            'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 
                                            'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 
                                            'change', 'Med']

In [5]:
trainX = pd.get_dummies(trainX, columns = array)
testX = pd.get_dummies(testX, columns = array)

In [6]:
trainX

,admission_typeid,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,glipizide-metformin_Steady,glimepiride-pioglitazone_No,metformin-rosiglitazone_No,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,change_Ch,change_No,Med_No,Med_Yes
0,1,1,7,4,44,0,15,4,0,0,...,0,1,1,0,1,0,0,1,0,1
1,5,3,17,3,15,1,20,0,0,0,...,0,1,1,0,1,0,0,1,0,1
2,3,1,1,1,45,1,18,0,0,1,...,0,1,1,0,1,0,0,1,0,1
3,5,3,1,3,52,0,14,12,1,0,...,0,1,1,0,1,0,0,1,1,0
4,1,3,7,5,62,0,12,0,0,2,...,0,1,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,2,1,7,2,53,3,17,0,0,0,...,0,1,1,0,1,0,1,0,0,1
71232,3,1,1,4,23,1,8,0,0,0,...,0,1,1,0,1,0,1,0,0,1
71233,2,3,7,6,70,0,23,0,0,0,...,0,1,1,0,1,0,0,1,1,0
71234,2,6,20,4,61,1,18,0,0,0,...,0,1,1,0,1,0,0,1,0,1


In [7]:
standardScaler = StandardScaler()
columns_to_scale = ['admission_typeid', 'discharge_disposition_id', 'admission_source_id', 
                    'time_in_hospital', 'num_procedures', 'num_medications', 'number_outpatient', 
                    'number_emergency', 'number_inpatient', 'number_diagnoses']
trainX[columns_to_scale] = standardScaler.fit_transform(trainX[columns_to_scale])
testX[columns_to_scale] = standardScaler.fit_transform(testX[columns_to_scale])

In [8]:
pca = PCA(n_components = 20)
x_pca = pd.DataFrame(pca.fit_transform(trainX))
xt_pca = pd.DataFrame(pca.fit_transform(testX))

In [9]:
select = sklearn.feature_selection.SelectKBest(k=20)
selected_features = select.fit(x_pca, trainY)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [trainX.columns[i] for i in indices_selected]
xtrainselect = trainX[colnames_selected]

C:\Users\Nadeem Khan\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [10]:
ax, ay, bx, by = train_test_split(x_pca, trainY, test_size=0.2, random_state=65)

In [11]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_features = None ,criterion='entropy' ,bootstrap=True,oob_score=True)

model.fit(ax, bx)
p = model.predict(ay)
score = accuracy_score(by, p)
score

<ipython-input-11-8fd9ac261be2>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(ax, bx)


0.5691325098259404

In [21]:
p

array(['NO', 'NO', 'NO', ..., '>5', 'NO', 'NO'], dtype=object)

In [13]:
p = model.predict(xt_pca)

In [15]:
df = pd.DataFrame(p)
df.to_csv('result.csv')

In [17]:
xtrainselect.columns

Index(['admission_typeid', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'cast_?',
       'cast_AfricanAmerican', 'cast_Asian', 'cast_Caucasian', 'cast_Hispanic',
       'cast_Other', 'gender_Female', 'gender_Male', 'gender_Unknown/Invalid'],
      dtype='object')

In [23]:
df


,0
0,NO
1,NO
2,NO
3,NO
4,>5
...,...
30525,>5
30526,NO
30527,>5
30528,NO
